# Comparison of Neural Models 2

Following an experiment to compare neural models, we discovered odd results showing that bag of words could outperform embeddings. This experiment attempts to tweak the embeddings to show the expected results under the assumption that the problem is not the amount of data. If the problem is the amount of data we will investigate this in another experiment.

Again, we will show Bag of Words results:

In [1]:
from scripts import training_helpers
from scripts.cross_validate import run_cross_validate
from tensorflow.keras.preprocessing.text import text_to_word_sequence, Tokenizer
from tensorflow.keras.activations import relu, sigmoid
from tensorflow.keras import Sequential
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Dropout, Flatten, Embedding
from tensorflow.keras.regularizers import l2

import gensim
import numpy as np

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

Using TensorFlow backend.


In [2]:
data_frame = training_helpers.get_data_frame()
raw_features = data_frame['review']
labels = [x for x in data_frame['deceptive']]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(raw_features)
bow_features = tokenizer.texts_to_matrix(raw_features, mode='tfidf')

In [ ]:
def get_ff_bow_model():
  model = Sequential([
      Dense(16, activation=relu, input_shape=(num_words,), kernel_regularizer=l2(0.01)),
      Dropout(0.25),
      Dense(8, activation=relu, kernel_regularizer=l2(0.01)),
      Dense(1, activation=sigmoid)
  ])
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  return model

ff_bow_scores = run_cross_validate(get_ff_bow_model, bow_features, labels, cv=10, verbose=0)

In [3]:
word_vectors = gensim.models.KeyedVectors.load_word2vec_format("../data/GoogleNews-vectors-negative300.bin", binary=True)

In [4]:
embedding_length = word_vectors.vector_size

In [5]:
# TODO Are we sure this should be post? should I use tokenizer.text_to_sequences? Should I have maxlen here?
padded_feature_vectors = pad_sequences(feature_vectors, padding='post', maxlen=embedding_length)

NameError: name 'feature_vectors' is not defined

In [ ]:
padded_feature_vectors.shape

In [6]:
corpus_words = tokenizer.word_index
corpus_vocab_size = len(corpus_words)+1
embedding_matrix = np.zeros((corpus_vocab_size, embedding_length))
for word, index in corpus_words.items():
  if word in word_vectors.vocab:
    embedding_matrix[index] = np.array(word_vectors[word], dtype=np.float32)

In [7]:
def get_ff_wv_model():
  model_ff_wv = Sequential([
      Embedding(corpus_vocab_size, embedding_length, weights=[embedding_matrix], trainable=False,
                input_length=corpus_vocab_size),
      Flatten(),
      Dense(16, activation=relu, kernel_regularizer=l2(0.01)),
      Dropout(0.25),
      Dense(8, activation=relu, kernel_regularizer=l2(0.01)),
      Dense(1, activation=sigmoid)
  ])
  model_ff_wv.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  return model_ff_wv

In [8]:
bow_features.shape

(1600, 9839)

In [9]:
ff_wv_scores = run_cross_validate(get_ff_wv_model, bow_features, labels, cv=10)

Fitting with:  (1440, 9839) labels (1440,)


ValueError: Error when checking input: expected embedding_input to have shape (300,) but got array with shape (9839,)